In [2]:
#%%
import numpy as np
import netCDF4 as nc
import os
import re
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap,BoundaryNorm
import matplotlib.cm as cm
from matplotlib import colors
import imageio
import pygeodesy
from pygeodesy.ellipsoidalKarney import LatLon
import folium
from shapely.geometry import Point,Polygon
from folium.plugins import FastMarkerCluster

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.ticker as mticker
from cartopy.io import shapereader
from sklearn.metrics import mean_squared_error
import matplotlib.colors as mcolors

In [4]:
#%% read output of RMSE
readpath = '/work/a06/yingying/Code/plot/online/jupyter/'
def read_wlv(exp,dahour_str):
    data = np.load(readpath+exp+dahour_str+'/wlv_rmse.npy')
    return data
def read_dis(exp,dahour_str):
    data = np.load(readpath+exp+dahour_str+'/dis_nrmse.npy')
    return data
wlv_ref = read_wlv('exp_ils','03')
dis_ref = read_dis('exp_ils','03')

wlv_ils06 = read_wlv('exp_ils','06')
dis_ils06 = read_dis('exp_ils','06')

wlv_ils12 = read_wlv('exp_ils','12')
dis_ils12 = read_dis('exp_ils','12')

wlv_man03 = read_wlv('exp_man','03')
dis_man03 = read_dis('exp_man','03')

wlv_man06 = read_wlv('exp_man','06')
dis_man06 = read_dis('exp_man','06')

wlv_man12 = read_wlv('exp_man','12')
dis_man12 = read_dis('exp_man','12')

In [5]:
#%% read the location of observations
file_alloc = 'gauge_good_wlv.txt'
# read allocate information
id_info = list()
ix      = list()
iy      = list()
with open('/work/a06/yingying/obs/alloc/'+file_alloc,'r') as file:
    alloc_info = file.readlines()
    for i in range(1,len(alloc_info)):
        id_info = id_info+[int(alloc_info[i].split()[0])]
        ix      = ix+[int(alloc_info[i].split()[8])-1]
        iy      = iy+[int(alloc_info[i].split()[9])-1]
id_info  = np.array(id_info)
ilon     = np.array(ix)
ilat     = np.array(iy)

In [6]:
def read_prep(file):
    nf = nc.Dataset(file,'r')
    varname = nf.variables.keys()
    varname = list(varname)
    print(varname)
    lat = np.array(nf.variables[varname[1]][:])
    lon = np.array(nf.variables[varname[2]][:])
    var = np.array(nf.variables[varname[3]][:])
    var = var*3600 # unit: mm/h 
    var = np.array(np.where(var<-900,np.nan,var))
    return lat,lon,var
lat,lon,rainf = read_prep('/work/a06/yingying/ils_data/tej_rain.nc')


['time', 'lat', 'lon', 'Precip']


In [16]:
#%% draw the map
def draw_diff(exp,dahour_str,var_dir,situ_rmse,vmin,vmax):
    fig = plt.figure(dpi = 600,figsize=(7.5,4.5))
    ax1 = fig.add_axes([0.1,0.1,0.8,0.8],projection=ccrs.PlateCarree())
    ax1.set_axis_off()
    # ax1.outline_patch.set_linewidth(0.35)
    ax1.set_extent([128,149,30,46], ccrs.PlateCarree())
    gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                    linewidth=1, color='gray', alpha=0, linestyle='-.')
    ax1.coastlines(alpha=1.,linestyle='-',color = "#3F3A3A",lw=0.8,resolution='10m')
    
    # mask the other place
    region_lon1 = [128.06, 128.06,138, 138 ]
    region_lat1 = [41 , 45.95 , 45.95, 41  ]
    region_lon2 = [128.06,128.06,131.4,131.4 ]
    region_lat2 = [34.5,41,41,34.5]
    ax1.fill(region_lon1, region_lat1, color='white', transform=ccrs.PlateCarree(), zorder=10)
    ax1.fill(region_lon2, region_lat2, color='white', transform=ccrs.PlateCarree(), zorder=10)
    
    gl.top_labels   = False
    gl.right_labels = False
    gl.bottom_labels   = False
    gl.left_labels = False
    norm = mcolors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
    con_plot2 = ax1.scatter(lon[ilon],lat[ilat],c=situ_rmse,s=1.5,zorder=2,cmap='coolwarm',transform=ccrs.PlateCarree(),vmin=vmin,vmax=vmax,alpha=0.8)
    # station highlight  
    ax1.scatter(lon[918],lat[556],c='black',alpha=0.5,zorder=3,transform=ccrs.PlateCarree(),marker='*',s=200)
    def color_bar(l,b,w,h):
      rect = [l,b,w,h]
      # cbar_ax = fig.add_axes(rect)
      return cbar_ax
    [l1,b1,w1,h1] = [0.63,0.12,0.01,0.25]
    [l2,b2,w2,h2] = [0.63,0.32,0.01,0.25]
    # cbar_ax2 = color_bar(l2,b2,w2,h2)

    # cb1.ax.set_title('NRMSE \n $(m^{3}/s)$',fontdict=font_label)
    # cb2 = plt.colorbar(con_plot2, cax=cbar_ax2,orientation="vertical",shrink=0.5)
    # ax1.set_title(title,loc='center')
    # plt.show()
    plt.savefig('/work/a06/yingying/Code/plot/online/jupyter/'+exp+dahour_str+var_dir+'.jpg', format='jpg',dpi=600)
    plt.close()
draw_diff('ils','06','wlv',wlv_ils06-wlv_ref,-0.5,0.5)
draw_diff('ils','12','wlv',wlv_ils12-wlv_ref,-0.5,0.5)
draw_diff('man','03','wlv',wlv_man03-wlv_ref,-0.5,0.5)
draw_diff('man','06','wlv',wlv_man06-wlv_ref,-0.5,0.5)
draw_diff('man','12','wlv',wlv_man12-wlv_ref,-0.5,0.5)

draw_diff('ils','06','dis',dis_ils06-wlv_ref,-0.2,0.2)
draw_diff('ils','12','dis',dis_ils12-wlv_ref,-0.2,0.2)
draw_diff('man','03','dis',dis_man03-wlv_ref,-0.2,0.2)
draw_diff('man','06','dis',dis_man06-wlv_ref,-0.2,0.2)
draw_diff('man','12','dis',dis_man12-wlv_ref,-0.2,0.2)

In [11]:
#%% interactive map
def plot_interactive(loc_row,disC_mean,disA_mean,situ_rmse,peak_time_obs):
    situ_loc = np.zeros((np.shape(lon_plot)[0],2))
    # lat,lon
    situ_loc[:,0] = lat[ilat]
    situ_loc[:,1] = lon[ilon]
    lat_cen = 40
    lon_cen = 141
    my_map = folium.Map(location=(lat_cen,lon_cen),width=1000,height=600, zoom_start=10,attr="Map tiles by Stamen Design, under CC BY 3.0. Data by OpenStreetMap, under ODbL.")
    
    cmap = plt.cm.coolwarm
    num_points = np.shape(lon_plot)[0]
    norm = plt.Normalize(-1,1,num_points-1)              
    
    # Save the map to an HTML file
    ind = 0
    for coord in situ_loc:
        # read related data
        loc_lat = ilat[ind]
        loc_lon = ilon[ind]
        loc_id  = id_info[ind]
        #obs
        obs_path = '/work/a06/yingying/MLIT/data/data_'+var_dir+'_hour2019/'
        filename = obs_path+str(loc_id)+'.bin'
        if os.path.exists(filename):   
            with open(filename, 'rb') as f:
                obs = np.fromfile(f, dtype=np.float64)
                obs = np.where(obs<-900,np.nan,obs)
                obs = np.where(obs>10**6,np.nan,obs)
            f.close()
    
        # read data
            if var_name == 'outflw':
                loc_sim = disC_mean[0:time_len,loc_lat,loc_lon]
                loc_obs = obs[stime:stime+time_len]
                loc_da  = disA_mean[0:time_len,loc_lat,loc_lon]
            else:
                loc_sim = disC_mean[0:time_len,loc_lat,loc_lon]-simmean[loc_lat,loc_lon]
                loc_obs = obs[stime:stime+time_len]-elvmean[loc_lat,loc_lon]
                loc_da  = disA_mean[0:time_len,loc_lat,loc_lon]-simmean[loc_lat,loc_lon]
            if np.isnan(peak_time_obs[ind]):
                loc_obs_str = 'None'
                loc_sim_str = 'None'
                loc_da_str = 'None'
            else:
                loc_obs_str = '{:.2f}'.format(loc_obs[int(peak_time_obs[ind])])
                loc_sim_str = '{:.2f}'.format(loc_sim[int(peak_time_obs[ind])])
                loc_da_str  = '{:.2f}'.format(loc_da[int(peak_time_obs[ind])])
            rmse_str = '{:.2f}'.format(situ_rmse[ind])
            color = colors.to_hex(cmap(norm(situ_rmse[ind])))
            if var_name == 'rivdph':
                pop = f"id:{loc_id}<br>loc_row: {loc_row[ind]}<br>delta_rmse: {rmse_str}<br>wlv_obs: {loc_obs_str}<br>wlv_sim: {loc_sim_str}<br>wlv_da: {loc_da_str}"
            else:
                pop = f"id:{loc_id}<br>loc_row: {loc_row[ind]}<br>delta_rmse: {rmse_str}<br>dis_obs: {loc_obs_str}<br>dis_sim: {loc_sim_str}<br>dis_da: {loc_da_str}"
            folium.CircleMarker(
                location=coord,
                popup=pop,
                radius=5,
                color=color,  # Assign color based on index
                fill=True,
                fill_color=color,
                fill_opacity=0.6,
            ).add_to(my_map)
        ind = ind+1
    my_map.save(exp+'_'+var_name+'_map.html')
    return my_map

my_map = plot_interactive(loc_row,disC_mean,disA_mean,situ_rmse,peak_time_obs)
my_map

In [50]:
simmean = np.nanmean(disC_mean,axis=0)

/tmp/ipykernel_440077/2889650537.py:1: RuntimeWarning: Mean of empty slice
  simmean = np.nanmean(disC_mean,axis=0)
